## Key numeric results
- Spectral (Khushaba) features — LDA 5-fold CV accuracy: **0.9085 ± 0.0013**
- Time-domain (MAV, WL, ZC, SSC) — LDA 5-fold CV accuracy: **0.8311 ± 0.0014**
- Combined (spectral + time) — LDA 5-fold CV accuracy: **0.9270 ± 0.0014**

## Commands executed (reproducible)
Below are the main commands used from the project root (using your `uv run` runner):
```
uv run pipeline.py S1_Male --out-dir results --feat khushaba --out S1_Male_features_khushaba.npz
uv run pipeline.py S1_Male --out-dir results --feat time --out S1_Male_features_time.npz
uv run combine_features.py --spec results/S1_Male_features_khushaba.npz --time results/S1_Male_features_time.npz --out results/S1_Male_features_combined.npz
uv run train.py --data results/S1_Male_features_khushaba.npz --models lda --cv 5
uv run train.py --data results/S1_Male_features_time.npz --models lda --cv 5
uv run train.py --data results/S1_Male_features_combined.npz --models lda --cv 5
```


In [3]:
# Inspect generated .npz files and show shapes / label distributions
import numpy as np
from pathlib import Path
files = {
    'spectral': Path('results/S1_Male_features_khushaba.npz'),
    'time': Path('results/S1_Male_features_time.npz'),
    'combined': Path('results/S1_Male_features_combined.npz'),
}
for name, p in files.items():
    if not p.exists():
        print(f'{name}: file missing -> {p}')
        continue
    data = np.load(p, allow_pickle=True)
    X = data['X']
    y = data['y']
    print(f'{name}: X.shape={X.shape}, y.shape={y.shape}, unique_labels={np.unique(y).shape[0]}')
    # show label counts for the first 8 classes (if present)
    unique, counts = np.unique(y, return_counts=True)
    print('  label distribution (first 8):', list(zip(unique[:8], counts[:8])))
    print()

spectral: file missing -> results/S1_Male_features_khushaba.npz
time: file missing -> results/S1_Male_features_time.npz
combined: file missing -> results/S1_Male_features_combined.npz


In [4]:
# Recorded CV results (as reported in the run logs)
cv_results = {
    'spectral_lda': (0.9085, 0.0013),
    'time_lda': (0.8311, 0.0014),
    'combined_lda': (0.9270, 0.0014),
}
for k, v in cv_results.items():
    mean, std = v
    print(f'{k}: {mean:.4f} ± {std:.4f}')

spectral_lda: 0.9085 ± 0.0013
time_lda: 0.8311 ± 0.0014
combined_lda: 0.9270 ± 0.0014


## Notes and next steps
- Combined features improved LDA accuracy vs spectral-only, suggesting time features add complementary information.
- Next analyses you might want:
  - Per-class precision/recall and confusion matrices (aggregated or per-fold).
  - Train and compare SVM, and an MLP baseline.
  - Feature ablation to quantify contribution of each time feature (MAV, WL, ZC, SSC).
  - Save trained models and export evaluation artifacts under `results/models/`.

To re-run the exact experiments, use the commands above from the project root.